In [5]:
import pandas as pd
import requests
import json
from getpass import getpass

In [16]:
url = 'https://natthawat.live/api'

# Read csv file

In [15]:
data={
    'การรับเข้านักศึกษา':pd.read_csv('admission.csv'),
    'หลักสูตร':pd.read_csv('curriculum.csv'),
    'ลงทะเบียนเรียน':pd.read_csv('enrollment.csv'),
    'คำถามทั่วไป':pd.read_csv('faq.csv'),
    # Remove imbalanced classes
    # 'ฝึกงาน': pd.read_csv('internship.csv'),
    # 'ทุนการศึกษา': pd.read_csv('scholarship.csv')
    }

# Set cookies and userId

In [35]:
username = input('Username: ')
password = getpass('Password: ')
login = {'username': username, 'password': password}
response = requests.post('%s/km/auth/login' % url,data=login)

if response.status_code == 201 or response.status_code == 200:
    cookies = response.cookies.get_dict()
    userId = json.loads(response.text)['id']
else:
    print("Username or password is not correct")

Username: root
Password: ········
Username or password is not correct


In [34]:
# Debug check
debug = False
if debug:
    print(cookies)
    print(userId)

# Create subcategory and category

In [39]:
query = False
for key,value  in data.items():
    category = key
    subcategories = value['Subcategory'].unique()

    createCategory = {
            'category': category
        }
    if query:
        response = requests.post('%s/km/category' % url, cookies=cookies, data=createCategory)
        categoryId = json.loads(response.text)['id']
    for subcategory in subcategories:
        createSubcategory = {
            'subcategory': subcategory,
            'categoryId': categoryId
            }
        if query:
            response = requests.post('%s/km/subcategory' % url, cookies=cookies, data=createSubcategory)
            subcategoryId = json.loads(response.text)['id']
        

# Get subcategory id and category id

In [40]:
response = requests.get('%s/km/category' % url, cookies=cookies)
categories = json.loads(response.text)

paramater
- categories -> Categories data
- categoryText -> category that want to search
- subcategoryText -> subcategory that want to search

return
- categoryId: string, subcategoryId: string

In [41]:
def findId(categories, categoryText, subcategoryText):
    for category in categories:
        if category['category'] == categoryText:
            categoryId = category['id']
            for subcategory in category['subcategories']:
                if subcategory['subcategory'] == subcategoryText:
                    subcategoryId = subcategory['id']
                    return categoryId, subcategoryId

# Create FAQ

In [43]:
query = False
for key,value  in data.items():
    category = key        
    questions = value['Questions']
    answers = value['Answers']
    subcategories = value['Subcategory']
    for i in range(len(questions)):
        categoryId, subcategoryId = findId(categories, category, subcategories[i])
        createFAQ={
            'question': questions[i],
            'answer': answers[i],
            'subcategoryId': subcategoryId,
            'lastEditorId': userId,
        }

        if query:
            response = requests.post('%s/km/faq' % url, cookies=cookies, data=createFAQ)

# Delete All

In [38]:
query = False
if query:
    # Remove faq
    response = requests.get('%s/km/faq' % (url), cookies=cookies)
    faqs=json.loads(response.text)
    for faq in faqs:
        response = requests.delete('%s/km/faq'  % (url), cookies=cookies, data=faq['id'])
    
    # Remove subcategory
    response = requests.get('%s/km/subcategory' % (url), cookies=cookies)
    subcategories=json.loads(response.text)
    for subcategory in subcategories:
        response = requests.delete('%s/km/subcategory'  % (url), cookies=cookies, data=subcategory['id'])
    
    # Remove category
    response = requests.get('%s/km/category' % (url), cookies=cookies)
    categories=json.loads(response.text)
    for category in categories:
        response = requests.delete('%s/km/category'  % (url), cookies=cookies, data=category['id'])

In [ ]:
query = False
column='category'
if query:
    # Remove faq
    response = requests.get('%s/km/%s' % (url,column), cookies=cookies)
    rows=json.loads(response.text)
    for row in rows:
        response = requests.delete('%s/km/%s'  % (url,column), cookies=cookies, data=row['id'])